# PUMAP


In [ ]:
%reload_ext autoreload
%autoreload 2

In [27]:
import torch
from umap_pytorch import PUMAP
import onnxruntime as ort

In [2]:
data = torch.randn((10000, 32))

pumap = PUMAP(
    encoder=None,  # nn.Module, None for default
    decoder=None,  # nn.Module, True for default, None for encoder only
    n_neighbors=10,
    min_dist=0.1,
    metric="euclidean",
    n_components=2,
    beta=1.0,  # How much to weigh reconstruction loss for decoder
    reconstruction_loss=torch.nn.functional.binary_cross_entropy_with_logits,  # pass in custom reconstruction loss functions
    random_state=None,
    lr=1e-3,
    epochs=10,
    batch_size=64,
    num_workers=1,
    num_gpus=1,
    match_nonparametric_umap=False,  # Train network to match embeddings from non parametric umap
)

pumap.fit(data)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Using accelerator: mps
Sun Apr 27 20:24:01 2025 Building RP forest with 10 trees
Sun Apr 27 20:24:02 2025 NN descent for 13 iterations
	 1  /  13
	 2  /  13
	 3  /  13
	 4  /  13
	 5  /  13
	 6  /  13
	 7  /  13
	 8  /  13
	 9  /  13
	Stopping threshold met -- exiting after 9 iterations



  | Name    | Type            | Params
--------------------------------------------
0 | encoder | default_encoder | 87.4 K
--------------------------------------------
87.4 K    Trainable params
0         Non-trainable params
87.4 K    Total params
0.350     Total estimated model params size (MB)


MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.


/Users/rcurrie/cell-space/venv/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 11 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.
MPS is available. Using Apple Silicon GPU.


`Trainer.fit` stopped: `max_epochs=10` reached.


In [3]:
embedding = pumap.transform(data)  # (50000, 2)

Reducing array of shape torch.Size([10000, 32]) to (10000, 2)


In [ ]:
embedding[0:10]

array([[ 0.09608545,  1.4405851 ],
       [-1.2620441 ,  0.61724925],
       [ 0.22486758, -0.16201767],
       [ 0.13304317, -1.3588085 ],
       [ 0.45416737,  2.266372  ],
       [-0.6763597 , -0.6924829 ],
       [ 0.77972907, -0.9883174 ],
       [ 0.92042553, -1.5424372 ],
       [ 1.1481138 , -2.1532557 ],
       [-0.8550528 ,  0.7312847 ]], dtype=float32)

In [25]:
torch.onnx.export(
    pumap.model.encoder.encoder,
    torch.zeros(1, data.shape[1]),
    "data/pumap_encoder.onnx",
    training=torch.onnx.TrainingMode.EVAL,
    input_names=["input"],
    output_names=["output"],
    export_params=True,
    opset_version=12,
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}},
)

In [28]:
session = ort.InferenceSession("data/pumap_encoder.onnx")
onnx_embedding = session.run(
    ["output"], {"input": data[0:10].to(torch.float32).detach().numpy()}
)

In [29]:
onnx_embedding

[array([[ 0.09608544,  1.440585  ],
        [-1.2620438 ,  0.6172491 ],
        [ 0.22486767, -0.16201784],
        [ 0.13304296, -1.3588084 ],
        [ 0.4541675 ,  2.2663722 ],
        [-0.67636   , -0.6924832 ],
        [ 0.779729  , -0.9883174 ],
        [ 0.9204255 , -1.5424374 ],
        [ 1.1481137 , -2.153256  ],
        [-0.8550529 ,  0.73128486]], dtype=float32)]

In [4]:
# if decoder enabled
recon = pumap.inverse_transform(embedding)  # (50000, 512)

TypeError: 'NoneType' object is not callable